In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-scores-and-betting-data/nfl_stadiums.csv
/kaggle/input/nfl-scores-and-betting-data/nfl_teams.csv
/kaggle/input/nfl-scores-and-betting-data/spreadspoke_scores.csv
/kaggle/input/nfl-scores-and-betting-data/spreadspoke.R


In [65]:
def clean_pfr_tables(input_df, afc_or_nfc):
    
    output_df = input_df.loc[~input_df.Tm.str.contains(afc_or_nfc)]
    output_df.Tm = output_df.Tm.str.replace('[^ 0-9a-zA-Z]+', '' ,regex=True)
    return output_df
    
def get_weekly_data(my_url):
    week_number = re.findall("(?:week=)(\d+)", my_url)[0]
    year_number = re.findall("(?:year=)(\d+)", my_url)[0]
    tables = pd.read_html(my_url)
    afc = tables[0]
    nfc = tables[1]
    
    cleaned_afc = clean_pfr_tables(afc, "AFC")
    cleaned_nfc = clean_pfr_tables(nfc, "NFC")
    
    combined_week = pd.concat([cleaned_afc, cleaned_nfc])
    
    combined_week['week_number'] = week_number    
    combined_week['year_number'] = year_number
    
    combined_week = combined_week.reset_index(drop=True)
    
    return combined_week

#weeks = list(range(1,19))
#years = list(range(2012,2022))
#df_list = []

#for year in years:
#    for week in weeks:
#        df_list.append(get_weekly_data(f'https://www.pro-football-reference.com/boxscores/standings.cgi?week={week}&year={year}&wk_league=NFL'))
        
#full_standings_data = pd.concat(df_list, axis=0, ignore_index = True)
#full_standings_data.to_csv("./full_standings_data.csv")

In [5]:
teams = pd.read_csv('../input/nfl-scores-and-betting-data/nfl_teams.csv')
teams.head()

,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,CRD,NFC,NaN,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,ARI,NFC,NaN,NFC,NFC East
3,Atlanta Falcons,Falcons,ATL,ATL,NFC,NFC South,NFC,NFC West
4,Baltimore Ravens,Ravens,BAL,RAV,AFC,AFC North,AFC,AFC Central


In [12]:
scores = pd.read_csv('../input/nfl-scores-and-betting-data/spreadspoke_scores.csv')
scores.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN


/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [64]:
afc = tables[0]
nfc = tables[1]

afc = afc.loc[~afc.Tm.str.contains("AFC")]
nfc = nfc.loc[~nfc.Tm.str.contains("NFC")]
afc.Tm = afc.Tm.str.replace('[^ 0-9a-zA-Z]+', '' ,regex=True)
nfc.Tm = nfc.Tm.str.replace('[^ 0-9a-zA-Z]+', '' ,regex=True)

week_number = re.findall("(?:week=)(\d+)", URL)[0]
year_number = re.findall("(?:year=)(\d+)", URL)[0]

combined = pd.concat([afc, nfc])

combined['week_number'] = week_number
combined['year_number'] = year_number

combined.reset_index(drop=True)

,Tm,W,L,T,W-L%,Pts,PtsO,PtDif,MoV,week_number,year_number
0,New England Patriots,1,0,0,1.000,34,13,21,1.3,1,2012
1,New York Jets,1,0,0,1.000,48,28,20,1.3,1,2012
2,Miami Dolphins,0,1,0,.000,10,30,-20,-1.3,1,2012
3,Buffalo Bills,0,1,0,.000,28,48,-20,-1.3,1,2012
4,Baltimore Ravens,1,0,0,1.000,44,13,31,1.9,1,2012
5,Cincinnati Bengals,0,1,0,.000,13,44,-31,-1.9,1,2012
6,Pittsburgh Steelers,0,1,0,.000,19,31,-12,-0.8,1,2012
7,Cleveland Browns,0,1,0,.000,16,17,-1,-0.1,1,2012
8,Houston Texans,1,0,0,1.000,30,10,20,1.3,1,2012
9,Indianapolis Colts,0,1,0,.000,21,41,-20,-1.3,1,2012
